In [34]:
batch_size = 32
BUFFER = 100
MONITOR = "val_accuracy"
BASE_EPOCHS = 100
BASE_PATIENCE = 3
MIN_DELTA = .02
LOGS = "logs"
DIR_OUT = "kt_out"
PROJECT = "kt_basics"
FACTOR = 3
VALIDATION = .2

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
  batch_size = 32
  !pip install keras_tuner
  !pip install wget

In [35]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras_tuner as kt
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Input, LeakyReLU
from keras.utils import np_utils
import datetime
from keras.applications.vgg16 import VGG16
from keras.models import Sequential

import os, warnings
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.data.experimental import sample_from_datasets

# Project 2 - Veggie Classification

For this assignment you'll need to classify some images of vegetables. 

## Parts

Please do two separate classifications:
<ol>
<li> First, create a model from scratch. 
<li> Use transfer learning to use a pretrained model of your choice, adapted to this data. 
</ol>

There won't be an explicit evaluation of accuracy, but you should take some steps to make each model as accurate as you reasonably can, any tuning option is fair game. Along with that, please structure it into a notebook that is well structured and clear that explains what you did and found. Think about:
<ul>
<li> Sections and headings. 
<li> A description of the approach taken (e.g. what did you do to determine size, tune, evaluate, etc...)
<li> Visualization of some important things such as a confusion matrix and maybe some images. 
<li> Results, mainly focused on the scoring of the test data. 
</ul>

The descriptions and explainations should highlight the choices you made and why you made them. Figure up to about a page or so worth of text total, explain what happened but don't write an essay. 

## Deliverables

Please sumbmit a link to your github, where everyhting is fully run with all the outputs showing on the page. As well, in the notebook please add some kind of switch controlled by a variable that will control if the notebook runs to train the model or to load the model in from weights - so I can download it and click run all, it will load the saved weights, and predict.

### Dataset

The code in the start of this notebook will download and unzip the dataset, and there is also a simple example of creating datasets. You can change the dataset bit to use a different approach if you'd like. The data is already split into train, validation, and test sets. Please treat the separate test set as the final test set, and don't use it for any training or validation. Each folder name is its own label.

### Evaluation

Marking will be based on the following:
<ul>
<li> Models are cretaed, tuned, and effective at classifying the data: 40%
<li> Descriptions and explanations of the approach taken: 20%
<li> Code is well structured and clear: 20%
</ul>

Overall the marking is pretty simple and direct, walk through the process of predicting the veggies, explain what you did, and show the results. If you do that, it'll get a good mark.

### Tips

Some hints that may be helpful to keep in mind:
<ul>
<li> The data is pretty large, so you'll want to use datasets rather than load everything into memory. The Keras docs have a few examples of different ways to load image data, our examples showed image generators and the image from directory datasets.  
<li> Be careful of batch size, you may hit the colab limits. 
<li> You'll want to use checkpoints so you can let it train and pick up where you left off.
<li> When developing, using a smaller dataset sample is a good idea. These weights could also be saved and loaded to jump start training on the full data. 
<li>

### Download and Unzip Data

In [36]:
def bar_custom(current, total, width=80):
    print("Downloading: %d%% [%d / %d] bytes" % (current / total * 100, current, total))
import wget
import zipfile
zip_name = "train.zip"

url = "https://jrssbcrsefilesnait.blob.core.windows.net/3950data1/vegetable_image_dataset.zip"

if not os.path.exists(zip_name):
    wget.download(url, zip_name, bar=bar_custom)

with zipfile.ZipFile(zip_name, 'r') as zip_ref:
    zip_ref.extractall()

In [89]:
# Generate Datasets - you can change this if desired
# ENSURE FILE PATHS MATCH CORRECTLY
IMAGE_SIZE=(224,224)
train_dir='Vegetable Images/train'
val_dir='Vegetable Images/validation'

# Load training data
train_ds = image_dataset_from_directory(
    train_dir,
    label_mode='categorical',
    image_size = IMAGE_SIZE,
    batch_size = None,
    shuffle = True
)

val_ds = image_dataset_from_directory(
    val_dir,
    label_mode='categorical',
    image_size = IMAGE_SIZE,
    batch_size = batch_size,
    shuffle = False
)

Found 15000 files belonging to 15 classes.
Found 3000 files belonging to 15 classes.


In [90]:
train_ds_sample = train_ds.take(2048)
print(train_ds_sample)

<TakeDataset element_spec=(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(15,), dtype=tf.float32, name=None))>


In [91]:
train_ds_sample

<TakeDataset element_spec=(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(15,), dtype=tf.float32, name=None))>

## Data Preparation

In [92]:
rescale = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255)
])

In [93]:
TPAIN = tf.data.AUTOTUNE


train_ds = train_ds.map(lambda x, y: (rescale(x), y))
train_ds = train_ds.batch(batch_size)
train_ds = train_ds.prefetch(buffer_size=TPAIN)

val_ds = val_ds.map(lambda x, y: (rescale(x), y))
val_ds = val_ds.prefetch(buffer_size=TPAIN)

In [94]:
train_ds_sample = train_ds_sample.map(lambda x, y: (rescale(x), y))
train_ds_sample = train_ds_sample.batch(batch_size)
train_ds_sample = train_ds_sample.prefetch(buffer_size=TPAIN)

## Custom Model Training

In [95]:
stopping = tf.keras.callbacks.EarlyStopping(monitor=MONITOR, patience=BASE_PATIENCE, mode="max", restore_best_weights=True)

In [96]:
def build_model(hp):
    model = Sequential()
    for i in range(hp.Int("num_layers", 1, 3)):
        model.add(layers.Conv2D(hp.Int("kernel_L"+str(i), min_value=32, max_value=128, step=16), (3, 3), input_shape=(224, 224, 3), padding=hp.Choice('padding', values=["same", "valid"])))
        model.add(layers.LeakyReLU())
        model.add(layers.MaxPooling2D())
    
    model.add(Flatten())
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.Dense(15, activation="sigmoid"))

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

In [97]:
file_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tuner = kt.BayesianOptimization(build_model,
                        objective=MONITOR,
                        max_trials=10,
                        directory=DIR_OUT+"/"+file_time,
                        project_name=PROJECT,
                        overwrite=True)

In [98]:
# tuner = kt.Hyperband(hypermodel = model_builder2,
#                      objective=MONITOR,
#                      max_epochs=BASE_EPOCHS,
#                      factor=3,
#                      directory=DIR_OUT,
#                      project_name=PROJECT, 
#                      overwrite=True)

In [99]:
# Get Results
tuner.search(train_ds_sample, epochs=3, validation_data=val_ds, callbacks=[stopping])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

Trial 10 Complete [00h 00m 37s]
val_accuracy: 0.06666667014360428

Best val_accuracy So Far: 0.7559999823570251
Total elapsed time: 00h 06m 55s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [100]:
# Build best model
best_model.build(input_shape=(None, 224,224,3))
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 64)      1792      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 222, 222, 64)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 64)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 112)     64624     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 109, 109, 112)     0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 112)      0         
 2D)                                                    

In [101]:
# Helper to plot loss
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.legend()
  plt.grid(True)
  plt.show()

def plot_acc(history):
  plt.plot(history.history['accuracy'], label='accuracy')
  plt.plot(history.history['val_accuracy'], label='val_accuracy')
  plt.legend()
  plt.grid(True)
  plt.show()

Steps to find model structure:
- Start with upper end of 512 adding 2 hidden layers

In [102]:
# # Basic Model
# model = Sequential()
# model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
# model.add(MaxPooling2D((2, 2)))
# model.add(Dropout(0.4))
# model.add(Conv2D(32, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Dropout(0.4))
# model.add(Conv2D(32, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Dropout(0.4))
# model.add(Conv2D(16, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))

# model.add(Flatten())
# model.add(Dense(16, activation='relu'))
# model.add(Dense(15, activation='softmax'))
# model.summary()

# model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=1e-3), metrics=["accuracy"])
# train_log = model.fit(train_ds, epochs=BASE_EPOCHS, verbose=1, callbacks=[stopping], validation_data=val_ds)

# plot_loss(train_log)
# plot_acc(train_log)

# val_acc_per_epoch = train_log.history['val_accuracy']
# best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
# print(f'Best epoch: {best_epoch:d}')
# print(f'Accuracy: {max(val_acc_per_epoch)}')

In [103]:
train_log = best_model.fit(train_ds, epochs=BASE_EPOCHS, verbose=1, callbacks=[stopping], validation_data=val_ds)

plot_loss(train_log)
plot_acc(train_log)

val_acc_per_epoch = train_log.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print(f'Best epoch: {best_epoch:d}')
print(f'Accuracy: {max(val_acc_per_epoch)}')

Epoch 1/100
469/469 [==============================] - 31s 66ms/step - loss: 0.5362 - accuracy: 0.8369 - val_loss: 0.5240 - val_accuracy: 0.8503
Epoch 2/100
469/469 [==============================] - 30s 64ms/step - loss: 0.1996 - accuracy: 0.9379 - val_loss: 0.5338 - val_accuracy: 0.8493
Epoch 3/100
469/469 [==============================] - 30s 64ms/step - loss: 0.1450 - accuracy: 0.9551 - val_loss: 0.7051 - val_accuracy: 0.8397
Epoch 4/100
469/469 [==============================] - 30s 64ms/step - loss: 0.1130 - accuracy: 0.9643 - val_loss: 0.7425 - val_accuracy: 0.8393
Epoch 5/100
469/469 [==============================] - 31s 65ms/step - loss: 0.1574 - accuracy: 0.9553 - val_loss: 0.9275 - val_accuracy: 0.8510
Epoch 6/100
469/469 [==============================] - 30s 63ms/step - loss: 0.1586 - accuracy: 0.9575 - val_loss: 1.6531 - val_accuracy: 0.8057
Epoch 7/100
135/469 [=======>......................] - ETA: 19s - loss: 0.2419 - accuracy: 0.9449

KeyboardInterrupt: 

## Transfer Learning Model

## Test Best Models and Illustrate Results

In [ ]:
test_dir='Vegetable Images/test'
test_ds = image_dataset_from_directory(
    test_dir,
    label_mode='categorical',
    image_size = IMAGE_SIZE,
    batch_size = batch_size,
)

Found 3000 files belonging to 15 classes.
